In [ ]:
%matplotlib inline

from scipy.odr import *
from scipy.stats import *
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import ast
from multiprocessing import Pool

import scipy

from IPython import display
from matplotlib.patches import Rectangle

from sklearn.metrics import mean_squared_error
import json

import scipy.stats as st
from sklearn.metrics import r2_score


from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

import copy

from sklearn.model_selection import LeaveOneOut, LeavePOut

from multiprocessing import Pool, cpu_count
import cv2

In [ ]:
dfAll = pd.read_pickle("PklData/AllData.pkl")
dfAll.head()

In [ ]:
df_actual = dfAll[(dfAll.Actual_Data == True) & (dfAll.Is_Pause == False)]
df_actual.head()

In [ ]:
print("all: %s, actual data: %s" % (len(dfAll), len(df_actual)))

In [ ]:
"""%%time
# this will be the faster repetition check
def is_max(row):
    return row.RepetitionID == df_filtered[(df_filtered.userID == row.userID) & (df_filtered.TaskID == row.TaskID) & (df_filtered.VersionID == row.VersionID)].RepetitionID.max()

pool = Pool(8)
df_filtered = df_actual.copy(deep=True)

max_vals = pool.map(is_max, [row for index, row in df_filtered.iterrows()])
df_filtered["IsMax"] = max_vals
print(df_filtered.head())
pool.close()"""

In [ ]:
%%time
df_filtered = df_actual.copy(deep=True)
for User in df_filtered.userID.unique():
    print(User)
    for Task in df_filtered.TaskID.unique():
        for Version in df_filtered.VersionID.unique():
            max_rep = df_filtered[(df_filtered.userID == User) & (df_filtered.TaskID == Task) & (df_filtered.VersionID == Version)].RepetitionID.max()
            df_filtered = df_filtered.drop(df_filtered[(df_filtered.userID == User) & (df_filtered.TaskID == Task) & (df_filtered.VersionID == Version) & (df_filtered.RepetitionID != max_rep)].index)

In [ ]:
df_filtered.to_pickle("PklData/dfFiltered.pkl")

In [ ]:
df_filtered = pd.read_pickle("PklData/dfFiltered.pkl")

In [ ]:
print("actual: %s, filtered data: %s" % (len(df_actual), len(df_filtered)))

In [ ]:
def detect_blobs(index, row):
    image = row.Image
    image = image.reshape(27, 15)
    temp, thresh = cv2.threshold(cv2.bitwise_not(image), 200, 255, cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [a for a in contours if cv2.contourArea(a) > 5 and cv2.contourArea(a) < 255]
    if len(contours) > 0:
        if (row.TaskID not in [6, 7, 23, 24]):
            # only one blob
            contours.sort(key=lambda a: cv2.contourArea(a))
            max_contour = contours[-1]
            xmax, ymax = np.max(max_contour.reshape(len(max_contour),2), axis=0)
            xmin, ymin = np.min(max_contour.reshape(len(max_contour),2), axis=0)
            croped_im = np.zeros((27,15))
            blob = image[max(ymin - 1, 0):min(ymax + 1, image.shape[0]),max(xmin - 1, 0):min(xmax + 1, image.shape[1])]
            croped_im[0:blob.shape[0],0:blob.shape[1]] = blob
            return (1, [croped_im])
        else:
            # two blobs
            blobs = []
            for i in range(1, 2):
                contours.sort(key=lambda a: cv2.contourArea(a))
                max_contour = contours[-1 * i]
                xmax, ymax = np.max(max_contour.reshape(len(max_contour),2), axis=0)
                xmin, ymin = np.min(max_contour.reshape(len(max_contour),2), axis=0)
                croped_im = np.zeros((27,15))
                blob = image[max(ymin - 1, 0):min(ymax + 1, image.shape[0]),max(xmin - 1, 0):min(xmax + 1, image.shape[1])]
                croped_im[0:blob.shape[0],0:blob.shape[1]] = blob
                blobs.append(blobs)
            return (2, blobs)
    else:
        return (0, np.zeros((27,15)))

In [ ]:
%%time
pool = Pool(cpu_count())
df_croped = df_filtered.copy()
temp_blobs = pool.starmap(detect_blobs, df_filtered.iterrows())
df_croped["BlobCount"] = [a[0] for a in temp_blobs]
df_croped["Blobs"] = [a[1] for a in temp_blobs]
df_croped.head()

In [ ]:
df_blobs = df_croped[df_croped.BlobCount >= 1]
print("detected blobs: %s" % len(df_blobs))
df_blobs.head()

In [ ]:
df_blobs.to_pickle("PklData/df_blobs_gestures.pkl")

# Build Gestures

In [ ]:
def detect_and_return_blobs(row):
    image = row[1].Image
    image = image.reshape(27, 15)
    temp, thresh = cv2.threshold(cv2.bitwise_not(image), 200, 255, cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [a for a in contours if cv2.contourArea(a) > 5 and cv2.contourArea(a) < 255]
    if len(contours) > 0:
        if (row[1].TaskID not in [6, 7, 23, 24]):
            # only one blob
            contours.sort(key=lambda a: cv2.contourArea(a))
            max_contour = contours[-1]
            xmax, ymax = np.max(max_contour.reshape(len(max_contour),2), axis=0)
            xmin, ymin = np.min(max_contour.reshape(len(max_contour),2), axis=0)
            croped_im = np.zeros((27,15))
            blob = image[max(ymin - 1, 0):min(ymax + 1, image.shape[0]),max(xmin - 1, 0):min(xmax + 1, image.shape[1])]
            croped_im[max(ymin - 1, 0):min(ymax + 1, image.shape[0]),max(xmin - 1, 0):min(xmax + 1, image.shape[1])] = blob
            return [croped_im]
        else:
            # two blobs
            blobs = []
            for i in range(1, 2):
                contours.sort(key=lambda a: cv2.contourArea(a))
                max_contour = contours[-1 * i]
                xmax, ymax = np.max(max_contour.reshape(len(max_contour),2), axis=0)
                xmin, ymin = np.min(max_contour.reshape(len(max_contour),2), axis=0)
                croped_im = np.zeros((27,15))
                blob = image[max(ymin - 1, 0):min(ymax + 1, image.shape[0]),max(xmin - 1, 0):min(xmax + 1, image.shape[1])]
                croped_im[max(ymin - 1, 0):min(ymax + 1, image.shape[0]),max(xmin - 1, 0):min(xmax + 1, image.shape[1])] = blob
                blobs.append(croped_im)
            return blobs
    else:
        return np.zeros((27,15))

In [ ]:
%%time
data_points = []

for UserID in df_blobs.userID.unique():
    print(UserID)
    for TaskID in df_blobs[df_blobs.userID == UserID].TaskID.unique():
        for VersionID in df_blobs[(df_blobs.userID == UserID) & (df_blobs.TaskID == TaskID)].VersionID.unique():
            for RepetitionID in df_blobs[(df_blobs.userID == UserID) & (df_blobs.TaskID == TaskID) & (df_blobs.VersionID == VersionID)].RepetitionID.unique():
                data = df_blobs[(df_blobs.userID == UserID) & (df_blobs.TaskID == TaskID) & (df_blobs.VersionID == VersionID) & (df_blobs.RepetitionID == RepetitionID)]
                gesture_im = np.zeros((27,15))
                for row in data.iterrows():
                    blobs = detect_and_return_blobs(row)
                    for blob in blobs:
                        gesture_im = gesture_im + blob
                #gesture_im = np.clip(gesture_im, 0, 255)
                data_points.append([UserID, TaskID, VersionID, RepetitionID, gesture_im])

print(len(data_points))

In [ ]:
df_gestures = pd.DataFrame(data_points)
df_gestures.columns = ["userID", "TaskID", "VersionID", "RepetitionID", "Gesture"]
df_gestures.head()

In [ ]:
df_gestures.to_pickle("PklData/df_gestures.pkl")

# display Gesture

In [ ]:
user_id = 4
len(df_gestures[(df_gestures.userID == user_id) & (df_gestures.TaskID == 15)])


In [ ]:
user_id = 4
task_id = 15
plt.clf()
for k in range(0, len(df_gestures[(df_gestures.userID == user_id) & (df_gestures.TaskID == task_id)])):
    data = df_gestures[(df_gestures.userID == user_id) & (df_gestures.TaskID == task_id)].Gesture.iloc[k]
    plt.figure(figsize=(6, 6))
    ax = plt.gca()
    plt.imshow(data, cmap='gray', vmin=0, vmax=255)
    # Loop over data dimensions and create text annotations.
    for i in range(0, data.shape[0]):
        for j in range(0, data.shape[1]):
            text = ax.text(j, i, int(data[i, j]),
                           ha="center", va="center", color="cyan", fontsize=1)
    plt.savefig("./GestureImages/gesture_%s_%s_%s.png" % (task_id, user_id, k), bbox_inches='tight')
    plt.show()